# Preprocessing .mat for NWB

In [2]:
import numpy as np
import h5py
import os
import yaml
import pandas as pd
from datetime import datetime
import shutil
import importlib
import numpy as np
import pandas as pd
import LO_NWB_conv



# Load data 

## mat

In [ ]:
import scipy.io
import h5py

WRAO039 = "data/mouse_anastasia/WR/AO039_20190626.mat"  
WRAO027 = "data/mouse_anastasia/WR/AO027_20181101.mat"
WnonRAO084 = "data/mouse_anastasia/WnonR/AO084_20210507.mat"
WnonRAO086 = "data/mouse_anastasia/WnonR/AO086_20210617.mat"

with h5py.File(WRAO039, 'r') as f:
    data_group = f['Data']
    data_ref = f["#refs#"]
    #print("Références trouvées :")
    #for key in data_ref.keys():
    #    print(f" - {key} → shape: {data_ref[key].shape}, dtype: {data_ref[key].dtype}")
    print("Contenu de 'Data' :")
    for key in data_group.keys():
        print(f" - {key} → shape: {data_group[key].shape}, dtype: {data_group[key].dtype}")


In [ ]:
with h5py.File(WRAO039, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	WRAO039 = {key: data_group[key][()] for key in data_group.keys()}

with h5py.File(WRAO027, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	WRAO027 = {key: data_group[key][()] for key in data_group.keys()}

with h5py.File(WnonRAO084, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	WnonRAO084 = {key: data_group[key][()] for key in data_group.keys()}

with h5py.File(WnonRAO086, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	WnonRAO086 = {key: data_group[key][()] for key in data_group.keys()}

In [ ]:
# Comparaison WRAO039 et WRAO027
keys_WRAO039 = set(WRAO039.keys())
keys_WRAO027 = set(WRAO027.keys())

same_keys = keys_WRAO039 == keys_WRAO027
print("Les deux dictionnaires ont les mêmes clés :", same_keys)

if not same_keys:
    print("Clés uniquement dans WRAO039 :", keys_WRAO039 - keys_WRAO027)
    print("Clés uniquement dans WRAO027 :", keys_WRAO027 - keys_WRAO039)

In [ ]:
# Comparaison WnonRAO084 et WnonRAO086
keys_WnonRAO084 = set(WnonRAO084.keys())
keys_WnonRAO086 = set(WnonRAO086.keys())

same_keys = keys_WnonRAO084 == keys_WnonRAO086
print("Les deux dictionnaires ont les mêmes clés :", same_keys)

if not same_keys:
    print("Clés uniquement dans WnonRAO084 :", keys_WnonRAO084 - keys_WnonRAO086)
    print("Clés uniquement dans WnonRAO086 :", keys_WnonRAO086 - keys_WnonRAO084)

In [ ]:
# Comaraison WR et WnonR
keys_WR = set(WRAO039.keys())
keys_WnonR = set(WnonRAO084.keys())
same_keys = keys_WR == keys_WnonR
print("Les deux dictionnaires ont les mêmes clés :", same_keys)

if not same_keys:
    print("Clés uniquement dans WR :", keys_WR - keys_WnonR)
    print("Clés uniquement dans WnonR :", keys_WnonR - keys_WR)


In [ ]:
if "Non" in subject_info.get("Session Type", "Unknown").strip():
    Rewarded = False
else:
    Rewarded = True

In [ ]:
with h5py.File(WRAO039, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	data = {key: data_group[key][()] for key in data_group.keys()}

## CSV


In [ ]:

csv_data = pd.read_csv("data/Subject_Session_Selection.csv", sep=";")
csv_data.columns = csv_data.columns.str.strip() 
csv_data.columns


In [ ]:

mouse = ''.join(chr(c) for c in WnonRAO084['mouse'].flatten())
date = ''.join(chr(c) for c in WnonRAO084['date'].flatten())
session_name = f"{mouse}_{date}"  

try:
    subject_info = csv_data[csv_data['Session'].astype(str).str.strip() == session_name].iloc[0]
except IndexError:
    raise ValueError(f"Session {session_name} not found in the CSV file.")


 # TOTAL _-_-_-_-_-_-_-_
 

In [13]:
# Supprimer tous les fichiers dans le dossier NWB output
for file in os.listdir("data/nwb_output"):
    file_path = os.path.join("data/nwb_output", file)
    if os.path.isfile(file_path):
        os.remove(file_path)
print("Tous les fichiers ont été supprimés du dossier data/nwb_output.")

filename = "data/mouse_anastasia/WR/AO027_20181101.mat" # Chemin vers le fichier .mat
importlib.reload(LO_NWB_conv)
nwb_path = LO_NWB_conv.convert_data_to_nwb_an(mat_file=filename, output_folder="data/nwb_output", output_folder_config="data")

Tous les fichiers ont été supprimés du dossier data/nwb_output.
**************************************************************************
-_-_-_-_-_-_-_-_-_-_-_-_-_-_- NWB conversion _-_-_-_-_-_-_-_-_-_-_-_-_-_-_
 
📃 Creating config file for NWB conversion :
   - data/AO027_20181101_config.yaml
 
📑 Created NWB file
     o 📌 Add general metadata
         - Subject metadata
         - Session metadata
         - Device metadata
         - Extracellular electrophysiology metadata
     o 📶 Add acquisition container
     o 🧠 Add units container
     o ⚙️ Add processing container
         - Behavior data
         - No ephys data for AN sessions
 
🔎 Validating NWB file before saving...
     o ✅ File is valid, no errors detected.
 
💾 Saving NWB file
     o 📂 NWB file saved at:
         - data/nwb_output/AO027_20181101_134512_2025_07_15.11-48-20.nwb
**************************************************************************
